In [1]:
# !pip install accelerate --upgrade
# !pip install -U bitsandbytes

In [24]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import itertools

import numpy as np
import pandas as pd

from santa.metrics import PerplexityCalculator

In [6]:
df = pd.read_csv("./data/sample_submission.csv")
sub_df = df.copy()
df

,id,text
0,0,advent chimney elf family fireplace gingerbrea...
1,1,advent chimney elf family fireplace gingerbrea...
2,2,yuletide decorations gifts cheer holiday carol...
3,3,yuletide decorations gifts cheer holiday carol...
4,4,hohoho candle poinsettia snowglobe peppermint ...
5,5,advent chimney elf family fireplace gingerbrea...


In [7]:
df["text"].map(lambda x: len(x.split(" ")))

0     10
1     20
2     20
3     30
4     50
5    100
Name: text, dtype: int64

In [10]:
model_path   = "google/gemma-2-9b"
max_length   = None
load_in_8bit = False
device_map   = "auto"

In [12]:
scorer = PerplexityCalculator(
    model_path,
    load_in_8bit=load_in_8bit,
    device_map=device_map,
)

Loading checkpoint shards: 100% 8/8 [00:09<00:00,  1.13s/it]


In [15]:
for i in range(len(df)):
    ret = scorer.get_perplexity(sub_df.loc[i, "text"])
    print(ret)

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


3884.785154167184
6055.995975407292
1118.666570066371
1287.5159346005714
353.15434961550477
355.0732086869962


In [22]:
def greedy_pair_merge(text, batch_size=8):
    tokens = text.split(" ")
    while len(tokens) > 1:
        print(f"# of tokens: {len(tokens)}")
        combs = list(itertools.permutations(tokens, 2))
        print(f"# of combinations: {len(combs)}")
        subs = pd.DataFrame({
            "id": np.arange(len(combs)),
            "text": list(map(lambda x: " ".join(x), combs)),
        })
        ret = scorer.get_perplexity(subs["text"].tolist(), batch_size=batch_size)
        best_comb = combs[np.argmin(ret)]
        print(f"best combination: {best_comb}")
        print(f"best ppl: {np.min(ret)}")
        for token in best_comb:
            if token in tokens:
                tokens.remove(token)
        tokens += [" ".join(best_comb)]
    return tokens[0]

In [26]:
target_id = 0
text = df.loc[target_id, "text"]
best_text = greedy_pair_merge(text)

# of tokens: 10
# of combinations: 90
best combination: ('scrooge', 'mistletoe')
best ppl: 3936.1031796399357
# of tokens: 9
# of combinations: 72
best combination: ('reindeer', 'scrooge mistletoe')
best ppl: 2445.89051545161
# of tokens: 8
# of combinations: 56
best combination: ('reindeer scrooge mistletoe', 'elf')
best ppl: 1518.2556904645223
# of tokens: 7
# of combinations: 42
best combination: ('reindeer scrooge mistletoe elf', 'gingerbread')
best ppl: 1200.0350101038055
# of tokens: 6
# of combinations: 30
best combination: ('reindeer scrooge mistletoe elf gingerbread', 'ornament')
best ppl: 892.2977498548082
# of tokens: 5
# of combinations: 20
best combination: ('reindeer scrooge mistletoe elf gingerbread ornament', 'advent')
best ppl: 879.2945765201335
# of tokens: 4
# of combinations: 12
best combination: ('reindeer scrooge mistletoe elf gingerbread ornament advent', 'family')
best ppl: 877.4613003242043
# of tokens: 3
# of combinations: 6
best combination: ('reindeer scroog

In [27]:
best_text

'reindeer scrooge mistletoe elf gingerbread ornament advent family fireplace chimney'